# Data Visualization Project

## Airplane Crashes Since 1908

***

Authors **Catarina Lourenço**

Student Numbers **C17709355**

Module **Big Data Concepts**

Lecturer **Abhishek Kaushik**

Course-Year **DT302-2 Business Analytics**

Submitted **31-03-2019**

***


## Summary
Data Visualization Project using Python and other tools, exploring [open data about aviation crashes](https://www.kaggle.com/saurograndi/airplane-crashes-since-1908), sourced from Kaggle.

*Python installation version:* anaconda3-5.1.0

*Additional packages:* geopy

*Other tools:* Google Fusion Tables, Excel


### Open data source selected: 
[Aviation crashes dataset](https://www.kaggle.com/saurograndi/airplane-crashes-since-1908)

#### Database Format
| Column Name | Summary |
|---------------|------------------------------------------------------|
| Date:         | Date of accident,  in the format - January 01, 2001 |
| Time:         | Local time, in 24 hr. format unless otherwise specified |
| Airline/Op:   | Airline or operator of the aircraft |
| Flight #:     | Flight number assigned by the aircraft operator |
| Route:        | Complete or partial route flown prior to the accident |
| AC Type:      | Aircraft type |
| Reg:          | ICAO registration of the aircraft |
| cn / ln:      | Construction or serial number / Line or fuselage number |
| Aboard:	    | Total aboard (passengers / crew) |
| Fatalities:	| Total fatalities aboard (passengers / crew) |
| Ground:	    | Total killed on the ground |
| Summary:	    | Brief description of the accident and cause if known |

### Final dataset used for analysis 

The original dataset wasn't used *as-is*, as we had to change some of its structure and contents in order to prepare it for the kind of analysis we had proposed ourselves to perform. 

**Data cleaning** involved mainly enforcing a coherent format within each column, decluttering text from words like "near" to keep only the relevant information, replacing NaN and strings by the correct numeric literal.

**Additions** Two columns for GPS coordinates, Latitude and Longitude. This was done using the geopy library (explain further in another section...) to scrap data from BingMaps. The script used can be found [here](add_coordinates.py).

**Deletions** The columns *Flight #*, *Route*, *Registration*, *cn/In* were dropped as we didn't require them for our analysis. Additionally, three rows for which the entries couldn't be verified were also dropped.

[This is the dataset](dataset/clean_v1_Airplane_Crashes_and_Fatalities_Since_1908.csv) that we used to start cleaning and preparing the data using pandas - See Part 1. Data Preparation.

[This is the final dataset](dataset/dataframe_exported.csv) that we used to proceed with analysis - See Part 2. Exploratory Data Analysis.

### Questions to be answered: 

**Descriptive:**
* Fatalities before and after hitting the ground
* Where do most planes crash (by time period)? Heat map with locations - period of 5-10 years? - change over time - animation/several time periods (WW2)?
* Separate into sections? Location by continent, night/day flight, type of aircraft etc
* Is there a specific make/model that appears more frequently than others
* Are they mainly passenger airplanes/cargo airplanes
* Does the age of the plane matter,ie, an older/newer plane regarding crashes

**Predictive:**
* When/where should we expect the next crash to happen? With how many fatalities?

**Text analysis:**
* Text analysis: find causes in description text
* (Advanced) Text analysis - sentiment analysis

## 1. Data Preparation

Cleaning, completing, dropping, converting

## 2. Exploratory Data Analysis

Main insights

## 3. Predictive Analysis

## Conclusions